<a href="https://colab.research.google.com/github/achett/mlflow/blob/main/MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow
!pip install statsmodels
!pip install numpy
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.3 MB/s eta 0:00:00


In [2]:
import os
import mlflow
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = 'achett2'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '2af8bb9bfaeec663848b89f850e29831f075854c'

mlflow.set_tracking_uri('https://dagshub.com/achett2/my-first-repo.mlflow')

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(url, usecols=[1], engine='python')
data = data.values.flatten()  # Convert dataframe to 1D array

# Split data into train and test sets
split = int(len(data) * 0.80)
train, test = data[0:split], data[split:len(data)]

# Set the MLflow experiment name
mlflow.set_experiment('Time Series ARIMA with statsmodels')

# Start an MLflow run
with mlflow.start_run():
    # Log dataset statistics
    mlflow.log_param("Dataset size", len(data))
    mlflow.log_param("Training size", len(train))
    mlflow.log_param("Test size", len(test))

    # Fit an ARIMA model
    model = ARIMA(train, order=(5,2,0))  # Modify order as appropriate for the dataset
    model_fit = model.fit()

    # Log model parameters
    mlflow.log_param("ARIMA order", "(5,2,0)")

    # Make predictions
    predictions = model_fit.forecast(steps=len(test))

    # Calculate and log mean squared error
    mse = mean_squared_error(test, predictions)
    mlflow.log_metric("MSE", mse)

    # Save the model to file
    model_fit.save("model_arima.pkl")
    mlflow.log_artifact("model_arima.pkl", "model")

    # Log the sklearn model and register as version 1
    mlflow.statsmodels.log_model(
        statsmodels_model=model_fit,
        artifact_path="stats-model",
        registered_model_name="arima_model2",
    )

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'arima_model2'.
2024/04/22 00:35:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: arima_model2, version 1
Created version '1' of model 'arima_model2'.


In [10]:
#################
# BUILD DOCKER CONTAINER
#################
# !mlflow models build-docker --model-uri "models:/arima_model/2" --name "arima_model_mlops" --enable-mlserver

run_id = '2048cfbcec474b33b32cb27e7e2dc3df'
image_name = "arima_model_mlops"
mlflow.models.build_docker(
    model_uri=f"https://dagshub.com/achett2/my-first-repo.mlflow/#/models/arima_model/versions/1",
    name=image_name,
    enable_mlserver=True,
)

ValueError: not enough values to unpack (expected 2, got 1)

In [7]:
#################
# VIEW MODELS
#################
from mlflow.tracking import MlflowClient
from pprint import pprint

client = MlflowClient()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1713745529578,
    'description': '',
    'last_updated_timestamp': 1713745750547,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1713745750547, current_stage='None', description='', last_updated_timestamp=1713745750547, name='arima_model', run_id='2048cfbcec474b33b32cb27e7e2dc3df', run_link='', source='mlflow-artifacts:/39c361fdc23d4d1aa2d568056b49e217/2048cfbcec474b33b32cb27e7e2dc3df/artifacts/stats-model', status='READY', status_message='', tags={}, user_id='', version='2'>],
    'name': 'arima_model',
    'tags': {}}
{   'aliases': {},
    'creation_timestamp': 1713746137964,
    'description': '',
    'last_updated_timestamp': 1713746138341,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1713746138341, current_stage='None', description='', last_updated_timestamp=1713746138341, name='arima_model2', run_id='fc2ffe06621746e6a024eda0f56b0a2e', run_link='', source='mlflow-artifacts:/39c361f